In [1]:
import tensorflow as tf
from tensorflow.keras.layers import *
from tensorflow.keras.utils import to_categorical, plot_model
from tensorflow.keras.models import Model
import pandas as pd
import numpy as np

In [76]:
# Load data
df_train = pd.read_csv('avocado-train.csv', index_col='id')
df_train = df_train.dropna(axis=0)

# Convert strings to numbers
types = {'conventional': 0, 'organic': 1}
df_train.type = [types[x] for x in df_train.type]
regions = df_train.region.unique()
regionDict = {x: i for i, x in enumerate(regions)}
df_train.region = [regionDict[x] for x in df_train.region]

types = df_train.pop('type')
regions = df_train.pop('region')

types = to_categorical(types)
regions = to_categorical(regions)
print(types.shape)
print(regions.shape)

df_train.drop(['XLargeBags'], axis=1, inplace=True)


def clean_date(x):
    return float(x.replace('-',''))


df_train['Date'] = df_train['Date'].apply(clean_date)

df_train = pd.concat([df_train, pd.DataFrame(types), pd.DataFrame(regions)], axis=1)
df_train.astype(float, copy=False)
y_train = df_train.pop('AveragePrice')

# from sklearn.preprocessing import MinMaxScaler
# x_scaler = MinMaxScaler(feature_range=(0, 1))  # scale between 0 and 1
# x_scaler.fit(df_train)
#
# y_scaler = MinMaxScaler(feature_range=(0, 1))
# y_scaler.fit(y_train.values.reshape(-1, 1))

# df_train = x_scaler.transform(df_train)
x_train = df_train.copy()

(12774, 2)
(12774, 54)


In [99]:
xIn = Input(shape=(x_train.shape[1],))
x = Dense(128, activation='swish')(xIn)
xOut = Dense(1, activation='linear')(x)

model = Model(inputs=xIn, outputs=xOut)
model.compile(optimizer=tf.keras.optimizers.Adam(1e-2), loss='mse', metrics=['mse'])
model.summary()

Model: "model_14"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_15 (InputLayer)       [(None, 65)]              0         
                                                                 
 dense_41 (Dense)            (None, 128)               8448      
                                                                 
 dense_42 (Dense)            (None, 1)                 129       
                                                                 
Total params: 8,577
Trainable params: 8,577
Non-trainable params: 0
_________________________________________________________________


In [100]:
callbacks = [tf.keras.callbacks.ReduceLROnPlateau(monitor='mse', factor=0.1, patience=15, verbose=1),
             tf.keras.callbacks.EarlyStopping(monitor='mse', patience=40, verbose=1, restore_best_weights=True)]


model.fit(np.asarray(x_train), np.asarray(y_train), epochs=100, batch_size=64, shuffle=True, callbacks=callbacks, use_multiprocessing=True)

Epoch 1/100
200/200 [==============================] - 1s 2ms/step - loss: 99978813440.0000 - mse: 99978813440.0000 - lr: 0.0100
Epoch 2/100
200/200 [==============================] - 1s 3ms/step - loss: 8046774272.0000 - mse: 8046774272.0000 - lr: 0.0100
Epoch 3/100
200/200 [==============================] - 0s 1ms/step - loss: 8140194304.0000 - mse: 8140194304.0000 - lr: 0.0100
Epoch 4/100
200/200 [==============================] - 0s 1ms/step - loss: 2850561280.0000 - mse: 2850561280.0000 - lr: 0.0100
Epoch 5/100
200/200 [==============================] - 0s 1ms/step - loss: 220919520.0000 - mse: 220919520.0000 - lr: 0.0100
Epoch 6/100
200/200 [==============================] - 0s 1ms/step - loss: 176458352.0000 - mse: 176458352.0000 - lr: 0.0100
Epoch 7/100
200/200 [==============================] - 0s 1ms/step - loss: 64453644.0000 - mse: 64453644.0000 - lr: 0.0100
Epoch 8/100
200/200 [==============================] - 0s 1ms/step - loss: 237881568.0000 - mse: 237881568.0000 - lr:

In [101]:
# Load data
df_test = pd.read_csv('avocado-test.csv', index_col='id')
df_test = df_test.dropna(axis=0)

# Convert strings to numbers
types = {'conventional': 0, 'organic': 1}
df_test.type = [types[x] for x in df_test.type]
regions = df_test.region.unique()
regionDict = {x: i for i, x in enumerate(regions)}
df_test.region = [regionDict[x] for x in df_test.region]

types = df_test.pop('type')
regions = df_test.pop('region')

types = to_categorical(types)
regions = to_categorical(regions)
print(types.shape)
print(regions.shape)

df_test.drop(['XLargeBags'], axis=1, inplace=True)


def clean_date(x):
    return float(x.replace('-',''))


df_test['Date'] = df_test['Date'].apply(clean_date)

df_test = pd.concat([df_test, pd.DataFrame(types), pd.DataFrame(regions)], axis=1)
df_test.astype(float, copy=False)

# df_test = pd.DataFrame(x_scaler.transform(df_test))

(5475, 2)
(5475, 54)


,DataBatch,Date,TotalVolume,4046,4225,4770,TotalBags,SmallBags,LargeBags,0,...,44,45,46,47,48,49,50,51,52,53
0,21.0,20160731.0,28969.34,80.77,27361.91,0.00,1526.66,1526.66,0.00,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,19.0,20170820.0,322962.89,5632.85,242365.82,2441.56,72522.66,22942.48,46083.51,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,49.0,20170122.0,155334.45,3657.79,74068.65,0.00,77608.01,27302.29,50305.72,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,12.0,20151004.0,10231.74,341.89,8519.00,0.00,1370.85,303.33,1067.52,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,10.0,20151018.0,10652.02,6905.95,39.52,0.00,3706.55,3706.55,0.00,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5470,8.0,20171105.0,10865.72,1262.85,2499.88,52.94,7050.05,6880.75,169.30,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
5471,47.0,20150201.0,57424.15,1321.47,30243.88,14.24,25844.56,25793.45,51.11,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5472,8.0,20171105.0,1319642.14,61830.21,632293.37,600.50,624918.06,563910.07,60984.25,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5473,14.0,20160918.0,9663.48,220.83,194.94,22.98,9224.73,9220.50,4.23,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [102]:
preds = model.predict(np.asarray(df_test))
# preds = y_scaler.inverse_transform(preds)
preds = pd.DataFrame(preds)
output = pd.DataFrame({'id': list(range(len(df_test))), 'AveragePrice': np.squeeze(preds)})
output.to_csv('avocado-submission-dnn.csv', index=False)
print(output)

        id  AveragePrice
0        0      1.331390
1        1      3.128998
2        2      1.982269
3        3      1.289398
4        4      1.023773
...    ...           ...
5470  5470      1.044281
5471  5471      0.954925
5472  5472      0.362101
5473  5473      1.490295
5474  5474      1.184128

[5475 rows x 2 columns]


In [103]:
# Download CTFSG Grader Libraries
import urllib.request, os
urllib.request.urlretrieve('https://raw.githubusercontent.com/alttablabs/ctfsg-utils/master/pyctfsglib.py', './pyctfsglib.py')
print('Downloaded pyctfsglib.py:', 'pyctfsglib.py' in os.listdir())
# Connect to graders
import sys
# insert at 1, 0 is the script path (or '' in REPL)
sys.path.insert(1, 'C:/Users/alien/Documents/PyCharm Projects/Cyberthon 2021/pyctfsglib.py')
import pyctfsglib as ctfsg
import random

USER_TOKEN = "WrlLCkymxwtgFwRHZsdmKfSwcdqIpnqoXEtRkciVRZJfBJUgcEJoxVZjNTQRdqkR" # You need to fill this up
GRADER_URL = random.choice([
  "http://chals.cyberthon22t.ctf.sg:50101/",
  "http://chals.cyberthon22t.ctf.sg:50102/"
])

grader = ctfsg.DSGraderClient(GRADER_URL, USER_TOKEN)
grader.submitFile('avocado-submission-dnn.csv')

Downloaded pyctfsglib.py: True
DSGraderClient: Successfully Connected!
[SERVER] MOTD: CHECK your USER_TOKEN and GRADER_URL HTTP address! I'm AVOCADO_PRICE @366389bec30e
ProofOfWork Challenge =>  ('CTFSGRB6778d6e15d2feba854cf27fdfa281023', 22)
ProofOfWork Answer Found! =>  28645939


'{"challenge":{"name":"Avocado Pricing"},"id":"cl21rhuaabo1r0763a9v07hl7","status":"PARTIALLY_CORRECT","multiplier":0,"submittedBy":{"username":"hci-69"},"createdAt":"2022-04-16T11:14:18Z"}'